lo mismo aca, tratar de juntar toda la info en un solo etl general de yelp. decidir en grupo como empezar que poner etc

#### **(Nota de Agustín):**

#### Propongo seguir la siguiente estructura ordenada para el proceso ETL de los datos de YELP como para los datos de Google

- ##### Agrupar ``yelp_1`` y ``yelp_2`` en este jupyter para una mejor consistencia.
- ##### Markdowns descriptivos.

###### Estructura estándar debajo:
---

### **Proceso ETL para el dataset YELP**

#### **Carga de Datos:**
- ``business.pkl``: Contiene información del comercio, incluyendo localización, atributos y categorías.

- ``checkin.json``: Registros en el negocio.
- ``review.json``: Contiene las reseñas completas, incluyendo el user_id que escribió el review y el business_id por el cual se escribe la reseña
- ``tip.json``: Tips (consejos) escritos por el usuario. Los tips son más cortas que las reseñas y tienden a dar sugerencias rápidas.
- ``user.parquet``: Data del usuario incluyendo referencias a otros usuarios amigos y a toda la metadata asociada al usuario.

#### **Business**


In [2]:
import pandas as pd

In [10]:
business = pd.read_pickle('Proyecto_final/Datos/business.pkl')


In [11]:
business.shape

(150346, 28)

In [12]:
business.isna().sum()

business_id          0
name                 0
address              0
city                 0
state                3
postal_code          0
latitude             0
longitude            0
stars                0
review_count         0
is_open              0
attributes       13744
categories         103
hours            23223
business_id     150341
name            150341
address         150341
city            150341
state           150341
postal_code     150341
latitude        150341
longitude       150341
stars           150341
review_count    150341
is_open         150341
attributes      150341
categories      150341
hours           150341
dtype: int64

#### El dataset contiene columnas duplicadas. Se eliminan esas columnas conservando solo las que contienen mayor cantidad de datos.

In [13]:
business = business.iloc[:,0:14]

In [23]:
business.columns

Index(['business_id', 'name', 'address', 'city', 'state', 'postal_code',
       'latitude', 'longitude', 'stars', 'review_count', 'is_open',
       'attributes', 'categories', 'hours'],
      dtype='object')

In [25]:
business.head(1) # Comprimí a uno para que se vea mejor. Pero a partir de head(25) ya se ven los Códigos Postales de otros países.

,business_id,name,address,city,state,postal_code,latitude,longitude,stars,review_count,is_open,attributes,categories,hours
0,Pns2l4eNsfO8kk83dixA6A,"Abby Rappoport, LAC, CMQ","1616 Chapala St, Ste 2",Santa Barbara,NaN,93101,34.426679,-119.711197,5.0,7,0,{'ByAppointmentOnly': 'True'},"Doctors, Traditional Chinese Medicine, Naturop...",None


### **SIGUIENDO ESTA ESTRUCTURA CONTINUAR TRASLADANDO LOS SCRIPTS DE YELP-1 Y YELP-2 AQUÍ...**

**(CORREGIR Y ELIMINAR CÓDIGOS POSTALES CON LETRAS PORQUE AL PARECER NO SON DE USA)**

### **Checkin**

El archivo JSON cargado contiene una columna `date` que tiene múltiples fechas anidadas en una sola celda, separadas por comas. Esto dificulta el análisis de los datos, ya que cada fecha debería estar en una fila separada para facilitar su manipulación y análisis.

Para resolver este problema, se decidió dividir las fechas en listas y luego expandir esas listas en filas separadas, manteniendo el `business_id` correspondiente para cada fecha.

1. **Dividir las fechas en listas**: Utilizamos el método `str.split(', ')` para dividir las fechas en listas.

2. **Expandir las listas en filas separadas**: Utilizamos el método `explode` para expandir las listas en filas separadas.

In [37]:
checkin = pd.read_json('Proyecto_final\Datos\checkin.json', lines=True)

In [38]:
checkin.shape

(131930, 2)

In [39]:
checkin.head()

,business_id,date
0,---kPU91CF4Lq2-WlRu9Lw,"2020-03-13 21:10:56, 2020-06-02 22:18:06, 2020..."
1,--0iUa4sNDFiZFrAdIWhZQ,"2010-09-13 21:43:09, 2011-05-04 23:08:15, 2011..."
2,--30_8IhuyMHbSOcNWd6DQ,"2013-06-14 23:29:17, 2014-08-13 23:20:22"
3,--7PUidqRWpRSpXebiyxTg,"2011-02-15 17:12:00, 2011-07-28 02:46:10, 2012..."
4,--7jw19RH9JKXgFohspgQw,"2014-04-21 20:42:11, 2014-04-28 21:04:46, 2014..."


In [40]:
print(checkin['date'].iloc[0])

2020-03-13 21:10:56, 2020-06-02 22:18:06, 2020-07-24 22:42:27, 2020-10-24 21:36:13, 2020-12-09 21:23:33, 2021-01-20 17:34:57, 2021-04-30 21:02:03, 2021-05-25 21:16:54, 2021-08-06 21:08:08, 2021-10-02 15:15:42, 2021-11-11 16:23:50


In [41]:
checkin.isna().sum()

business_id    0
date           0
dtype: int64

In [42]:
# Se dividen las fechas en listas
checkin['date'] = checkin['date'].str.split(', ')

# Se expanden las listas en filas separadas
checkin_exploded = checkin.explode('date')

# Resultado
print(checkin_exploded.head())
print(checkin_exploded.shape)

              business_id                 date
0  ---kPU91CF4Lq2-WlRu9Lw  2020-03-13 21:10:56
0  ---kPU91CF4Lq2-WlRu9Lw  2020-06-02 22:18:06
0  ---kPU91CF4Lq2-WlRu9Lw  2020-07-24 22:42:27
0  ---kPU91CF4Lq2-WlRu9Lw  2020-10-24 21:36:13
0  ---kPU91CF4Lq2-WlRu9Lw  2020-12-09 21:23:33
(13356875, 2)


In [43]:
## Guardar archivo con el siguiente código:

# checkin.to_parquet("datasets/yelp/yelp_checkin.parquet") 

### **Review**

#### El diccionario de datos indica que esta dataset contiene la columna `business_id` refiriendo al negocio en en dataset **Business**. Por lo tanto para proceder con este ETL hay que arreglar los errores de `estados` y `codigo postal` de **Business** para poder verificar ids únicos.

### **Tip**

In [3]:
tip = pd.read_json('Proyecto_final/Datos/tip.json', lines=True)

In [8]:
tip.shape

(908915, 5)

In [7]:
tip.head()

,user_id,business_id,text,date,compliment_count
0,AGNUgVwnZUey3gcPCJ76iw,3uLgwr0qeCNMjKenHJwPGQ,Avengers time with the ladies.,2012-05-18 02:17:21,0
1,NBN4MgHP9D3cw--SnauTkA,QoezRbYQncpRqyrLH6Iqjg,They have lots of good deserts and tasty cuban...,2013-02-05 18:35:10,0
2,-copOvldyKh1qr-vzkDEvw,MYoRNLb5chwjQe3c_k37Gg,It's open even when you think it isn't,2013-08-18 00:56:08,0
3,FjMQVZjSqY8syIO-53KFKw,hV-bABTK-glh5wj31ps_Jw,Very decent fried chicken,2017-06-27 23:05:38,0
4,ld0AperBXk1h6UbqmM80zw,_uN0OudeJ3Zl_tf6nxg5ww,Appetizers.. platter special for lunch,2012-10-06 19:43:09,0


In [9]:
tip.isna().sum()

user_id             0
business_id         0
text                0
date                0
compliment_count    0
dtype: int64

In [10]:
fecha_antigua = tip['date'].min()
print(f'tip mas antiguo: {fecha_antigua}')

tip mas antiguo: 2009-04-16 13:11:49


In [ ]:
# Código para guardar el archivo tip en formato parquet:

# tip.to_parquet("datasets/yelp/yelp_tip.parquet")

### **User**